Rajiv D. Rago

204178

December 12, 2021

In [16]:
## Problem A: POS Machine, improved
from tkinter import *
from tkinter import filedialog

class POSmachine:
    
    def __init__(self, name):
        self.window = Tk()
        self.window.title("My thing")
        
        self.priceList = []
        self.salesInvoice = {}
        self.salesInvoiceTracker = 0
        self.salesReport = {}
        self.salesReportTracker = 0
        self.subTotal = 0
        self.grandTotal = 0
        self.invoicesPrinted = 0
        self.name = name #name of store
        self.salesView = True
        self.labels = ["ID", "Product", "Unit Price", "Quantity", "Total Price"]
        
        set_price_list = Button(self.window, text = "Set the price list", command = self.setPriceList)
        set_price_list.pack()
        
        self.window.mainloop()
    
    def home(self):
        self.clearWindow(self.window)
        
        left_frame = Frame(self.window)
        left_frame.pack(side = LEFT)
        
        right_frame = Frame(self.window)
        right_frame.pack(side = RIGHT)
        
        set_price_list = Button(left_frame, text = "Set the price list", command = self.setPriceList)
        set_price_list.pack()
        
        products_container = LabelFrame(left_frame, height = 400, text = "Price List")
        products_container.pack()
        
        self.printPriceList(products_container)
        
        sales_container = LabelFrame(right_frame, height = 400, width = 500, text = "Sales Invoice")
        change_view = Button(right_frame, text = "View Sales Report")
        
        self.salesCanvas = self.createScrollableSection(sales_container)
        self.salesCanvasContents = {}
        
        #Source: obtained from: <https://www.w3schools.com/python/python_lambda.asp>
        #Date: December 12, 2021
        #Name of Author/Programmer: W3Schools
        change_view.configure(command = lambda frame = self.salesCanvas, label_frame = sales_container, button = change_view: self.changeSalesView(frame, label_frame, button))
        change_view.pack()
        sales_container.pack()
        
        for x in range(5):
            label = Label(self.salesCanvas, text = self.labels[x]).grid(row = 0, column = x)
        
        self.totalDisplay = Label(right_frame, text = "Total:")
        self.totalDisplay.pack(side = "left")
        
        export_btn2 = Button(right_frame, text = "Export Sales Report", command = lambda to_export = self.salesReport, name = "Sales Report": self.export(to_export, name))
        export_btn2.pack(side = "right")
        
    def setPriceList(self):
        self.export(self.salesInvoice, "invoice")
        self.subTotal = 0
        
        file_path = filedialog.askopenfilename(filetypes = (("Readable Files", "*.csv *.txt"), ("All Files", "*.*")))
        inputFile = open(file_path, "r")

        products = inputFile.read().split("\n")

        if file_path[-3:] == "csv": #parses CSV
            temp_list = [product.split(",") for product in products]

            inQuotes = False
            for item in temp_list:
                self.priceList.append([])
                for data in item:

                    if inQuotes: #handles quotes
                        self.priceList[-1][-1] += "," + data
                        inQuotes = False
                    else:
                        self.priceList[-1].append(data)
                        if data[0] == "\"":
                            inQuotes = True

        elif file_path[-3:] == "txt": #parses TXT
            self.priceList = [product.split("\t") for product in products]
        
        self.salesInvoice = {}
        inputFile.close()
        self.home()

    def createScrollableSection(self, frame):
        #Source: obtained from: <https://youtu.be/VmlgrrXAqb4>
        #Date: December 12, 2021
        #Name of Author/Programmer: CodeWorked
        canvas = Canvas(frame)
        canvas.pack(side=LEFT, fill = "both", expand = "yes")
        
        scrollbar = Scrollbar(frame, orient = VERTICAL, command = canvas.yview)
        scrollbar.pack(side = RIGHT, fill = "y") 
        
        canvas.configure(yscrollcommand = scrollbar.set)
        
        canvas.bind("<Configure>", lambda e: canvas.configure(scrollregion = canvas.bbox("all")))
        
        to_display = Frame(canvas)
        canvas.create_window((0,0), window = to_display, anchor = "nw")
        
        return to_display
    
    def printPriceList(self, frame):
        to_display = self.createScrollableSection(frame)
        data = {}
        
        quantity = Label(to_display, text = "Quantity").grid(row = 1, column = 3)
        
        for y in range(len(self.priceList)):
            product = self.priceList[y]
            product_amount = len(product)
            data[y] = []
            
            for x in range(product_amount):
                data[y].append(Label(to_display, text = product[x]))
                data[y][-1].grid(row = y+1, column = x)
                
            if y != 0:  
                #Source: obtained from: <https://www.geeksforgeeks.org/python-tkinter-validating-entry-widget/>
                #Date: <December 12, 2021>
                #Name of Author/Programmer: GeeksforGeeks
                vcmd = frame.register(self.isInt)
                data[y].append(Entry(to_display, width = 10, validate = "key", validatecommand = (vcmd, "%P")))
                data[y][-1].bind("<KeyRelease>", lambda x, widget = data[y][-1], ID = y: self.makeNewTransaction(ID, widget))
                data[y][-1].grid(row = y+1, column = product_amount)
    
    def isInt(self, inp):
        if inp.isdigit() or inp == "":
            return True
        else:
            return False
    
    def changeSalesView(self, frame, label_frame, button):
        self.clearWindow(frame)
        frame.configure(height = 400)
        self.salesCanvasContents = {}
        
        if self.salesView:
            button.configure(text = "View Sales Invoice")
            label_frame.configure(text = "Sales Report")
            self.salesView = False
            
            for x in range(5):
                label = Label(frame, text = self.labels[x]).grid(row = 0, column = x)
                
            self.printSales(frame, self.salesReport)
            
        else:
            button.configure(text = "View Sales Report")
            label_frame.configure(text = "Sales Invoice")
            self.salesView = True
            
            for x in range(5):
                label = Label(frame, text = self.labels[x]).grid(row = 0, column = x)
                
            self.printSales(frame, self.salesInvoice)

    def addProductToTransaction(self, ID, quantity):
        tempList = self.priceList[ID] + [quantity, quantity * int(self.priceList[ID][2])]
        
        if ID in self.salesInvoice: #happens if item is already on invoice
            if quantity == 0: #happens if input is 0
                self.salesInvoice[ID][2] = 0
                self.salesInvoice[ID][3] = 0
                
                self.salesReport[ID][2] -= self.salesInvoice[ID][-1]
                self.salesReport[ID][3] -= self.salesInvoice[ID][-1] * int(self.priceList[ID][2])
                
                self.subTotal -= self.salesInvoice[ID][-1] * int(self.priceList[ID][2])
                self.grandTotal -= self.salesInvoice[ID][-1] * int(self.priceList[ID][2])
                
            else: #happens normally
                quantity_diff = tempList[3] - self.salesInvoice[ID][-1]
                price_diff = (quantity_diff * int(self.priceList[ID][2]))
            
                self.salesInvoice[ID][2] += quantity_diff
                self.salesInvoice[ID][3] += price_diff
                
                self.salesReport[ID][2] += quantity_diff
                self.salesReport[ID][3] += price_diff
                
                self.subTotal += quantity_diff
                self.grandTotal += quantity_diff
        else:
            self.salesInvoice[ID] = tempList[1:]
            
            self.subTotal += tempList[-1]
            self.grandTotal += tempList[-1]
            
            if ID in self.salesReport:
                self.salesReport[ID][2] += tempList[3]
                self.salesReport[ID][3] += tempList[4]
            else:
                self.salesReport[ID] = tempList[1:]

        
        if len(self.salesInvoice[ID]) >= 5: #checks if text box already has an input
            self.salesInvoice[ID][-1] = quantity
        else:
            self.salesInvoice[ID].append(quantity)
        
        
        if self.salesView:
            self.editSales(self.salesCanvas, ID, self.salesInvoice[ID])
        else:
            self.editSales(self.salesCanvas, ID, self.salesReport[ID])
        
        if self.salesInvoice[ID][2] == 0: #removes the item if quantity is 0
            self.salesInvoice.pop(ID)
            
    def makeNewTransaction(self, ID, widget):
        quantity = widget.get()
        if quantity == "":
            self.addProductToTransaction(ID, 0)
        else:
            self.addProductToTransaction(ID, int(quantity))
    
    def printSales(self, frame, data):
        
        for ID, info in data.items():
            self.editSales(frame, ID, info)
            
    def editSales(self, frame, ID, data):
        if self.salesView:
            self.totalDisplay.configure(text = "Total: {}".format(self.subTotal))
        else:
            self.totalDisplay.configure(text = "Grand Total: {}".format(self.grandTotal))
            
        if ID in self.salesCanvasContents:
            for item in self.salesCanvasContents[ID]:
                item.destroy()
        if data[2] != 0:
            self.salesCanvasContents[ID] = [Label(frame, text = str(ID))]
            self.salesCanvasContents[ID][0].grid(row = ID, column = 0)
            
            for x in range(4):
                
                self.salesCanvasContents[ID].append(Label(frame, text = data[x]))
                self.salesCanvasContents[ID][-1].grid(row = ID, column = x + 1)
    def export(self, to_export, name):
        output = ""
        
        for ID, properties in to_export.items():
            line = str(ID)
            
            if type(properties[3]) != str:
                self.subTotal += properties[3]
            
            for item in properties:
                line += "\t{}".format(item)
            output += line + "\n"
        
        if "invoice" in name.lower():
            outputFile = open("{}{}.txt".format(name,self.salesInvoiceTracker), "w")
            self.salesInvoiceTracker += 1
        elif "report" in name.lower():
            outputFile = open("{}{}.txt".format(name,self.salesReportTracker), "w")
            self.salesReportTracker += 1
            
        outputFile.write(output)
        outputFile.close()
        
    def clearWindow(self, container):
        #removes everything
        for widget in container.winfo_children():
            widget.destroy()

    def quit(self):
        self.window.destroy()
    
rustans = POSmachine("Rustan's")